### Nota
Este código importa todos los archivos .csv que esten en la carpeta VehiculosExportados y los agrupa en un solo DF, contiene los datos de Vehículos Exportados del 2005-2021. 

In [1]:
import pandas as pd
import glob
import geojson
import geopandas as gpd

path = "VehiculosExportados" # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
countriesTable = pd.read_csv("ExportsMap/tc_pais_destino.csv")
exportsDF = pd.merge(frame, countriesTable, how="left", on=["ID_PAIS_DESTINO", "ID_PAIS_DESTINO"])


In [2]:
exportsDF.columns

Index(['PROD_EST', 'COBERTURA', 'ANIO', 'ID_MES', 'MARCA', 'MODELO', 'TIPO',
       'SEGMENTO', 'ID_PAIS_DESTINO', 'UNI_VEH', 'DESC_PAIS_DESTINO',
       'ENGLISH'],
      dtype='object')

In [39]:
exportsCountryYear= exportsDF.groupby(["ENGLISH", "ANIO"])["UNI_VEH"].sum().unstack().reset_index()
exportsMatrix = exportsCountryYear.fillna(0).rename_axis("", axis="columns").rename(columns={
    "ENGLISH":"country",
    2005:"exports2005",
    2006:"exports2006",
    2007:"exports2007",
    2008:"exports2008",
    2009:"exports2009",
    2010:"exports2010",
    2011:"exports2011",
    2012:"exports2012",
    2013:"exports2013",
    2014:"exports2014",
    2015:"exports2015",
    2016:"exports2016",
    2017:"exports2017",
    2018:"exports2018",
    2019:"exports2019",
    2020:"exports2020",
    2021:"exports2021",
})
exportsMatrix

,country,exports2005,exports2006,exports2007,exports2008,exports2009,exports2010,exports2011,exports2012,exports2013,exports2014,exports2015,exports2016,exports2017,exports2018,exports2019,exports2020,exports2021
0,Algeria,0.0,0.0,0.0,0.0,776.0,43.0,112.0,0.0,0.0,0.0,0.0,0.0,168.0,3639.0,3053.0,588.0,1.0
1,Angola,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,71.0,56.0,8.0,33.0
2,Anguilla,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,Antigua and Barbuda,4.0,1.0,1.0,0.0,5.0,2.0,4.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Argentina,6367.0,14717.0,17692.0,29780.0,25524.0,58483.0,69061.0,63021.0,61624.0,25720.0,39391.0,38994.0,40635.0,33129.0,5718.0,7338.0,4158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Uzbekistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
145,Venezuela,6067.0,10653.0,26153.0,3163.0,0.0,7.0,0.0,10.0,182.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,34.0
146,Vietnam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,262.0,70.0,26.0,244.0
147,Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,1260.0,56.0,11.0,0.0,34.0,0.0,0.0,4.0,0.0


In [44]:
worldGeoJson = gpd.read_file("ExportsMap/world.geojson")
worldGeoJson.rename(columns={'NAME':'country'}, inplace=True)
exportsMatrixGeoJSon = pd.merge(worldGeoJson, exportsMatrix, how="left", on=["country", "country"])
exportsMatrixGeoJSon

,country,NAME_LONG,ADM0_A3,ISO_A2,ISO_A3,WB_A2,WB_A3,geometry,exports2005,exports2006,...,exports2012,exports2013,exports2014,exports2015,exports2016,exports2017,exports2018,exports2019,exports2020,exports2021
0,Afghanistan,Afghanistan,AFG,AF,AFG,AF,AFG,"POLYGON ((74.89230 37.23110, 74.48730 37.22590...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,Angola,AGO,AO,AGO,AO,AGO,"MULTIPOLYGON (((23.96750 -10.87230, 24.01120 -...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,49.0,71.0,56.0,8.0,33.0
2,Albania,Albania,ALB,AL,ALB,AL,ALB,"POLYGON ((20.06500 42.54680, 20.24960 42.31860...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aland,Aland Islands,ALD,AX,ALA,-99,-99,"POLYGON ((19.91850 60.37020, 20.27760 60.27870...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,Andorra,AND,AD,AND,AD,ADO,"POLYGON ((1.70700 42.50280, 1.44790 42.43460, ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Zambia,Zambia,ZMB,ZM,ZMB,ZM,ZMB,"POLYGON ((32.92090 -9.40790, 32.97470 -9.60130...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,Zimbabwe,Zimbabwe,ZWE,ZW,ZWE,ZW,ZWE,"POLYGON ((30.39630 -15.63600, 30.40260 -16.001...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,Vatican,Vatican City,VAT,VA,VAT,VA,VAT,"POLYGON ((12.45832 41.90260, 12.45835 41.90154...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,San Marino,San Marino,SMR,SM,SMR,SM,SMR,"POLYGON ((12.44146 43.87457, 12.47505 43.89909...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
exportsGeoJson = exportsMatrixGeoJSon[['country', 'exports2005','exports2006','exports2007', 'exports2008','exports2009','exports2010','exports2011','exports2012','exports2013','exports2014','exports2015','exports2016','exports2017','exports2018','exports2019', 'exports2020', 'exports2021', 'geometry']]
exportsGeoJson.to_file("ExportsMap/exports.geojson", driver="GeoJSON")
exportsGeoJson

,country,exports2005,exports2006,exports2007,exports2008,exports2009,exports2010,exports2011,exports2012,exports2013,exports2014,exports2015,exports2016,exports2017,exports2018,exports2019,exports2020,exports2021,geometry
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((74.89230 37.23110, 74.48730 37.22590..."
1,Angola,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,71.0,56.0,8.0,33.0,"MULTIPOLYGON (((23.96750 -10.87230, 24.01120 -..."
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((20.06500 42.54680, 20.24960 42.31860..."
3,Aland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((19.91850 60.37020, 20.27760 60.27870..."
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((1.70700 42.50280, 1.44790 42.43460, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((32.92090 -9.40790, 32.97470 -9.60130..."
195,Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((30.39630 -15.63600, 30.40260 -16.001..."
196,Vatican,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((12.45832 41.90260, 12.45835 41.90154..."
197,San Marino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((12.44146 43.87457, 12.47505 43.89909..."


In [6]:
#exportsGroupbyCountry = exportsDF.loc[exportsDF["ANIO"] == 2020].groupby(exportsDF["ENGLISH"])["UNI_VEH"].sum().sort_values(ascending=False)
#exportsGroupbyDF = exportsGroupbyCountry.to_frame(name = 'exports').reset_index()
#exportsReady = exportsGroupbyDF.rename(columns={
#    "ENGLISH":"country",
#    "exports":"exportamount"
#})
#exportsJson = exportsReady.to_json(orient="records")